# Setup

In [1]:
!pip install lime

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import KFold, cross_validate, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, recall_score

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

# _

In [3]:
dataset = "kaggle"  # ["kaggle", "ibm", "mix"] # TODO: Implement mix strategy

### Data Loading

In [4]:
if dataset == "kaggle":
  numeric_features = ["MonthlyCharges",
                      'Age',
                      #'TotalCharges',
                      #"Tenure"
                      ]
  categorical_features = ['ContractType',
                          'InternetService'
                          ]
  binary_features = ["TechSupport",
                    "Churn",
                    #"Gender"
                    ]
  features = numeric_features + categorical_features + binary_features

  df = pd.read_csv('data/customer_churn_data.csv', usecols = features)
  if 'InternetService' in df.columns:
    df['InternetService'] = df['InternetService'].fillna('No')

In [5]:
if dataset == "ibm":
  numeric_features = [#'tenure',
                      'MonthlyCharges',
                      #'TotalCharges'
                      ]
  categorical_features = ['InternetService',
                          'Contract',
                          'PaymentMethod'
                          ]
  binary_features = [#'gender',
                    'SeniorCitizen',
                    'Partner',
                    'Dependents',
                    'PhoneService',
                    'OnlineSecurity',
                    'OnlineBackup',
                    'DeviceProtection',
                    'TechSupport',
                    'PaperlessBilling',
                    'StreamingTV',
                    'StreamingMovies',
                    'Churn',
                    'MultipleLines'
                    ]
  features = numeric_features + categorical_features + binary_features

  df = pd.read_csv('data/WA_Fn-UseC_-Telco-Customer-Churn.xls', usecols = features)
  if 'MultipleLines' in df.columns:
    df['MultipleLines'] = df['MultipleLines'].replace('No phone service', 'No')

## Cross-validation

In [6]:
df = pd.get_dummies(df, columns=categorical_features, drop_first=False, dtype=int)
df = pd.get_dummies(df, columns=binary_features, drop_first=True, dtype=int)

X = df.drop(columns=['Churn_Yes'])
y = df['Churn_Yes']

In [7]:
cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

metrics = ['accuracy', 'f1_weighted', 'precision', 'recall', 'roc_auc']
scores = {metric: [] for metric in metrics}

classifiers = {
    "NB": GaussianNB(),
    "LR": LogisticRegression(random_state=42),
    "SVM": SVC(random_state=42, probability=True),
    "DT": DecisionTreeClassifier(random_state=42),
    "RF": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42)
}

results = {}
for name, model in classifiers.items():
    scores = {metric: [] for metric in metrics}
    for train_index, test_index in cv.split(X, y):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]

        scaler = StandardScaler()
        X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
        X_test[numeric_features] = scaler.transform(X_test[numeric_features])

        resampler = RandomUnderSampler(random_state=42)
        X_train_balanced, y_train_balanced = resampler.fit_resample(X_train, y_train)

        #X_train_balanced = X_train
        #y_train_balanced = y_train

        model.fit(X_train_balanced, y_train_balanced)

        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1]

        scores['accuracy'].append(accuracy_score(y_test, y_pred))
        scores['f1_weighted'].append(f1_score(y_test, y_pred))
        scores['precision'].append(precision_score(y_test, y_pred))
        scores['recall'].append(recall_score(y_test, y_pred))
        scores['roc_auc'].append(roc_auc_score(y_test, y_prob))

    results[name] = {
        "Accuracy Mean": np.mean(scores['accuracy']),
        #"Accuracy Std": np.std(scores['accuracy']),
        "F1 Score Mean": np.mean(scores['f1_weighted']),
        #"F1 Score Std": np.std(scores['f1_weighted']),
        "Precision Mean": np.mean(scores['precision']),
        #"Precision Std": np.std(scores['precision']),
        "Recall Mean": np.mean(scores['recall']),
        #"Recall Std": np.std(scores['recall']),
        "AUC Mean": np.mean(scores['roc_auc']),
        #"AUC Std": np.std(scores['roc_auc']),
    }

results_df = pd.DataFrame(results).T
results_df = results_df.round(4)
results_df

,Accuracy Mean,F1 Score Mean,Precision Mean,Recall Mean,AUC Mean
NB,0.857,0.9119,1.0000,0.8381,0.9427
LR,0.851,0.9078,1.0000,0.8313,0.9408
SVM,0.854,0.9099,1.0000,0.8347,0.9510
DT,0.883,0.9310,0.9720,0.8936,0.8490
RF,0.893,0.9360,0.9925,0.8856,0.9333
XGBoost,0.881,0.9279,0.9947,0.8697,0.9381
